In [1]:
# coming soon

In [26]:
import pandas as pd
import ipywidgets as widgets
import numpy as np
import sys
sys.path.append("../../")
import os
import glob
import cv2

import plotly.graph_objs as go
import plotly.express as px
from ipywidgets import HBox, VBox, Button

In [27]:
log_directory = os.path.join("..", "logs")
csv_files = glob.glob(os.path.join(log_directory, "*.csv"))
filenames = [os.path.split(csv_file)[-1] for csv_file in csv_files]
opts = [csv_tuple for csv_tuple in zip(filenames, csv_files)]

In [28]:
csv_selector = widgets.Dropdown(
    options=opts,
    description='CSV Log file:',
    value="..\\logs\\ALG_005_EXP_001-PatchMatch-MacLean_et_al-Numba.csv"

)

available_metrix = ['abs_error',
       'mse', 'avg', 'eucledian', 'bad1', 'bad2', 'bad4', 'bad8']
metrics_selector = widgets.Dropdown(
    options=[(m,m) for m in available_metrix],
    description='CSV Log file:'
)

VBox([csv_selector, metrics_selector])

In [57]:
def read_image_binary(path):
    with open(path, "rb") as f:
        b = f.read()
        return b


selected_file = csv_selector.value
df = pd.read_csv(selected_file)
df = df.drop_duplicates()
df = df.sort_values(by=metrics_selector.value)

df["loaded_imgs"] = [read_image_binary(path) for path in df["image_filename"]]

In [58]:
from ipywidgets import Image, Layout

fig_a_image = Image(value=df["loaded_imgs"].iloc[0], 
                   layout=Layout(height='375px', width='450px'))
fig_b_image = Image(value=df["loaded_imgs"].iloc[0], 
                   layout=Layout(height='375px', width='450px'))

In [59]:

fig_a = go.FigureWidget(px.scatter(df, x=df['kernel_size'],
            y=df[metrics_selector.value]))

fig_b = go.FigureWidget(px.scatter(df, x=df['match'],
            y=df[metrics_selector.value]))

fig_a_data = fig_a.data[0]

fig_b_data = fig_b.data[0]



In [60]:
def hover_fn_a(trace, points, state):
    if(len(points.point_inds)>0):
        ind = points.point_inds[0]
        fig_a_image.value = df["loaded_imgs"].iloc[ind]


def hover_fn_b(trace, points, state):
    if(len(points.point_inds)>0):
        ind = points.point_inds[0]
        fig_b_image.value = df["loaded_imgs"].iloc[ind]

fig_a_data.on_hover(hover_fn_a)
fig_b_data.on_hover(hover_fn_b)

In [70]:
dashboard_a = HBox([fig_a, fig_a_image])
dashboard_b = HBox([fig_b, fig_b_image])

In [73]:
kernel_sizes= df["kernel_size"].unique()
kernel_specs = df["kernel_spec"].unique
match_values = (df["match"].min(), df["match"].max(), 10)
metrics = ['abs_error',
       'mse', 'avg', 'eucledian', 'bad1', 'bad2', 'bad4', 'bad8']

nonoccluded = [True, False]


from ipywidgets import interact
interact_dict = dict(scene = ["teddy", "cones"],
          match = match_values,
          metric = metrics,
          nonoccluded = nonoccluded)

@interact(
    **interact_dict
         )
def update_match(
          scene = "teddy",
          match = 10,
          metric = "bad2",
          nonoccluded = True):
    with fig_a.batch_update():
        nonoccluded = not nonoccluded
        temp_df = df[
            (df["match"]==match) &
            (df["scene"]==scene) &
            (df["are_occlusions_errors"] == nonoccluded)] 
        fig_a_data.x = temp_df["kernel_size"]
        fig_a_data.y = temp_df[metric]
        fig_b.update_layout(title="Match value w.r.t. "+metric,
        xaxis_title="match",
        yaxis_title=metric
        )
dashboard_a

interactive(children=(Dropdown(description='scene', options=('teddy', 'cones'), value='teddy'), IntSlider(valu…

    'data': [{'hovertemplate': 'kernel_size=%{x}<br>abs_error=%{y}<extra></extra…

In [63]:

@interact(scene = ["teddy", "cones"],
          kernel_size = kernel_sizes,
          metric = metrics,
          nonoccluded = nonoccluded
         )
def update_match(
          scene = "teddy",
          kernel_size = "3x3",
          metric = "bad2",
          nonoccluded = True):
    with fig_b.batch_update():
        nonoccluded = not nonoccluded
        temp_df = df[
            (df["kernel_size"]==kernel_size) &
            (df["scene"]==scene) &
            (df["are_occlusions_errors"] == nonoccluded)] 
        fig_b_data.x = temp_df["match"]
        fig_b_data.y = temp_df[metric]
        fig_b.update_layout(title="Kernel size w.r.t. "+metric,
        xaxis_title="match",
        yaxis_title=metric
        )
dashboard_b

interactive(children=(Dropdown(description='scene', options=('teddy', 'cones'), value='teddy'), Dropdown(descr…

    'data': [{'hovertemplate': 'match=%{x}<br>abs_error=%{y}<extra></extra>',
  …

In [64]:
fig_c = px.scatter(df, x=df['kernel_size'],
            y=df[metrics_selector.value], animation_frame="match")

In [65]:
fig_c

In [69]:
fig_c

AttributeError: 'Figure' object has no attribute 'animation_frame'

In [ ]:
fig_d = px.scatter(df, x='match',
            y=metrics_selector.value, animation_frame="kernel_size")

In [67]:
fig_d